# Definicion de 50 Modelos de Automoviles

## 1. Modelos de las marcas mas vendidas en Peru
A partir de un dataset publico "us-car-models-data-master", se filtraron los modelos de acuerdo a la marca

In [ ]:
import os
import pandas as pd

# Directorio con los archivos CSV por año
csv_directory = r'us-car-models-data-master'

# Lista de marcas seleccionadas (en minúsculas)
selected_makes = [
    'toyota', 'hyundai', 'kia', 'chevrolet', 'changan',
    'nissan', 'suzuki', 'dfsk', 'jac', 'volkswagen',
    'mitsubishi', 'ford', 'chery', 'mazda', 'renault',
    'honda', 'great wall', 'subaru', 'mg', 'foton'
]

# DataFrame para acumular datos
filtered_data = pd.DataFrame()

# Leer y filtrar archivos por año
for year in range(1992, 2025):
    csv_file = os.path.join(csv_directory, f'{year}.csv')
    
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        df['make'] = df['make'].str.lower()
        df_filtered = df[df['make'].isin(selected_makes)].copy()
        df_filtered['year'] = year  # Agregar columna del año
        filtered_data = pd.concat([filtered_data, df_filtered], ignore_index=True)

# Guardar los datos filtrados en CSV
filtered_data.to_csv('new_models.csv', index=False)

# Crear resumen por marca
summary = (
    filtered_data
    .groupby('make')
    .agg(
        cantidad_modelos=('model', 'nunique'),
        años=('year', lambda x: ','.join(map(str, sorted(set(x)))))
    )
    .reset_index()
    .rename(columns={'make': 'marca'})
)

# Guardar el resumen en un archivo Excel
summary.to_excel('resumen_marcas.xlsx', index=False)


## 2. Pytrends para comparar el interes de los modelos 

### 2.1 Generacion de Archivos Excel con los intereses en el tiempo

In [ ]:
import pandas as pd
from pytrends.request import TrendReq
# Initialize Pytrends
pytrends = TrendReq(hl='es-PE')  # Set the region to Peru
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
df= pd.read_csv('models.csv')
makes = df['make'].unique().tolist()
models = df[['make', 'model']].values.tolist()
models = [' '.join(make_model) for make_model in models]
new_models = [models[102]] + models[:102] + models[102:]  #'toyota Corolla' : models[102]
new_models = list(dict.fromkeys(new_models))

In [ ]:
modelos_faltantes = ['changan X7 Plus', 'changan UNI-K', 'changan UNI-T', 'changan New CS55 Plus', 'changan New CS15', 'changan New CS35 Plus', 'changan New Van', 'changan HONOR S', 'changan Grand SUPERVAN', 'changan Grand VAN Turismo', 'changan New Alsvin', 'changan NEW F70',
'dfsk Glory 500', 'dfsk Glory iX5', 'dfsk DFSK E5', 'dfsk Glory 600', 'dfsk Glory 560', 'dfsk Glory 580', 'dfsk Glory 330', 'dfsk C37', 'dfsk K07S', 'dfsk K05S Cargo Panel', 'dfsk C31 Cabina Simple', 'dfsk C35 Cargo Panel',
'jac JS2', 'jac JS3', 'jac JS4', 'jac JS6', 'jac JS8', 'jac T6', 'jac T8', 'jac T8 PRO', 'jac Nueva Refine', 'jac Sunray', 'jac X200', 'jac E-JS1', 'jac E-JS4', 'jac E-T8', 'jac E-M3',
'renault New Kwid', 'renault Stepway', 'renault All New Duster', 'renault New Renault Kardian', 'renault New Koleos', 'renault New Kwid E Tech', 'renault New Oroch', 'renault Master Furgón', 'renault Master Minibus',
'chery Tiggo 8 Pro Hybrid', 'chery Tiggo 8 Pro Max', 'chery Tiggo 8 Pro', 'chery Tiggo 8', 'chery Tiggo 7 Pro Hybrid', 'chery Tiggo 7 Pro', 'chery Tiggo 4 Pro', 'chery Tiggo 2 Pro', 'chery Arrizo 5',
'jetour X70', 'jetour X70Plus', 'jetour Dashing', 'jetour X90Plus', 'jetour Traveller T2',
'geely GX3 Pro', 'geely New Coolray', 'geely New Okavango', 'geely New Starray', 'geely Emgrand',
'great wall H6 GT', 'great wall DARGO', 'great wall H6', 'great wall JOLION', 'great wall POER AT', 'great wall POER MT', 'great wall WINGLE 7 DIESEL', 'great wall WINGLE 5 DIESEL', 'great wall WINGLE 5 GASOLINA', 'great wall H6 HEV', 'great wall JOLION HEV',
'mg ZX', 'mg ZX PLUS', 'mg HS', 'mg RX5', 'mg ONE', 'mg RX8', 'mg 5', 'mg 6', 'mg GT', 'mg 4', 'mg 3', 'mg 3 HYBRID+',
'jinbei F50', 'jinbei H2L', 'jinbei Haise', 'jinbei Konect',
'foton K0', 'foton K1', 'foton K2', 'foton BigVan Toano', 'foton Tunland G7 4x4', 'foton Tunland G7 4x2', 'foton Tunland G9 4x4', 'foton Tunland G9 4x4', 'foton Tunland G9 DS']

new_list = [new_models[0]] + modelos_faltantes
new_list

In [ ]:
import pandas as pd
import time

# Initialize variables
data = []
previous_df = pd.DataFrame()

def calculate_means(chunk):
    pytrends.build_payload(chunk, cat=0, timeframe='today 5-y', geo='PE')
    data = pytrends.interest_over_time()
    return data.mean()

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

for chunk_number, chunk in enumerate([lst + [new_list[0]] for lst in list(chunks(new_list[1:], 4))]):
    if(chunk_number < 83): continue
    try:
        means = calculate_means(chunk)

        # Display relevant details in the console
        print(f"Processing chunk {chunk_number}")
        print("Chunk models:", chunk)

        # Save the current chunk's data
        current_data = []
        for i, model in enumerate(chunk):
            is_the_same_value = model == 'toyota Corolla'
            current_data.append([chunk_number, is_the_same_value, model, means[i]])

        current_df = pd.DataFrame(current_data, columns=['chunk_number', 'is_the_same_value', 'model', 'value'])

        # Save current and previous dataframes to Excel
        with pd.ExcelWriter(f'chunk_data_rest_{chunk_number}.xlsx') as writer:
            current_df.to_excel(writer, sheet_name='Current', index=False)
            previous_df.to_excel(writer, sheet_name='Previous', index=False)

        # Update the previous_df to be the current one for the next iteration
        previous_df = current_df.copy()

        time.sleep(60)

    except Exception as e:
        print(f"Error processing chunk {chunk_number}: {e}")
        break


### 2.2 Calculo de la media

In [ ]:
import pandas as pd
import os
import re

In [ ]:
# Directory containing the Excel files
directory = './respaldo/bkup1/'

# List to store the filtered values
all_values = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.xlsx'):
        # Extract chunk_number from the filename using regex
        match = re.search(r'chunk_data(?:_rest)?_(\d+)\.xlsx', filename)
        if match:
            chunk_number = int(match.group(1))
        print(chunk_number)
        # Read the Excel file
        filepath = os.path.join(directory, filename)
        df = pd.read_excel(filepath)
        
        # Filter rows where is_the_same_value is True
        filtered_df = df[df['is_the_same_value'] == True]
        
        # Append the filtered values to the list
        all_values.extend(filtered_df['value'].tolist())

# Calculate the mean of all filtered values
value_mean = pd.Series(all_values).mean()

# Display value_mean
print(value_mean)

# Save value_mean to a text file
with open('value_mean.txt', 'w') as file:
    file.write(str(value_mean))

In [ ]:
import pandas as pd
import os
import re

# Directory containing the Excel files
directory = './respaldo/bkup1/'

# List to store the filtered values
all_values = []

# Dictionary to store the mean value for each chunk
chunk_means = {}

#read the value_mean from the text file
with open('value_mean.txt', 'r') as file:
    mean_value = float(file.read())

# List to store the updated DataFrames
updated_dfs = []

# Second pass: Update the values and store the updated DataFrames
for filename in os.listdir(directory):
    if filename.endswith('.xlsx'):
        # Extract chunk_number from the filename using regex
        match = re.search(r'chunk_data(?:_rest)?_(\d+)\.xlsx', filename)
        if match:
            chunk_number = int(match.group(1))
        
        # Read the Excel file
        filepath = os.path.join(directory, filename)
        df = pd.read_excel(filepath)
         
        
        # Get the value of the row where is_the_same_value is True
        true_value = df[df['is_the_same_value'] == True]['value'].values[0]
        
        # Update the values
        df['value'] = df['value'] * mean_value / true_value
        
        # Append the updated DataFrame to the list
        updated_dfs.append(df)

# Concatenate all the updated DataFrames
result_df = pd.concat(updated_dfs, ignore_index=True)

# Save the result to a new Excel file
result_df.to_excel('updated_values.xlsx', index=False)

In [ ]:
#from update_values, give me the top 50 with the highest value and the model name is not repeated

result_df = pd.read_excel('updated_values.xlsx')

# Sort the DataFrame by value in descending order
sorted_df = result_df.sort_values(by='value', ascending=False)

# Drop duplicates based on the model column, keeping the first occurrence
unique_models_df = sorted_df.drop_duplicates(subset='model', keep='first')

# Select the top 50 rows
top_50_df = unique_models_df.head(50)

# Save the top 50 rows to a new Excel file
top_50_df.to_excel('top_50_values.xlsx', index=False)